In [55]:
import numpy as np
import pandas as pd
from numpy.linalg import solve, norm, cond
from scipy.linalg import hilbert

In [2]:
def lu(a): #алгоритм LU-декомпозиции
    n = a.shape[0]
    l = np.identity(n)
    u = np.zeros([n,n])
    for i in range(n):
        for j in range(n):
            if i <= j:
                u[i,j] = a[i,j]-sum([l[i,k]*u[k,j] for k in range(i)])
            else:
                l[i,j] = (a[i,j]-sum([l[i,k]*u[k,j] for k in range(j)]))/u[j,j]
    return l,u

In [45]:
def lu_solve(l,u,b=None): #решение СЛАУ LU-методом
    if b is None:
        b = np.random.uniform(-100,100,size=(u.shape[1]))
    y = np.zeros(l.shape[1])
    x = np.zeros(u.shape[1])
    n = len(x)
    for i in range(len(y)):
        y[i] = b[i] - sum([l[i,p]*y[p] for p in range(i)])
    for j in range(len(x)):
        x[n-j-1]=(y[n-j-1]-sum([u[n-j-1,n-p-1]*x[n-p-1] for p in range(j)]))/u[n-j-1,n-j-1]
    return x
            

Проверим, что решения, полученные lu-методом, совпадают с решением системы:

In [95]:
a = np.random.rand(2,2)
b = np.random.rand(2)

In [96]:
l,u = lu(a)
norm(solve(a,b)-lu_solve(l,u,b))

0.0

In [56]:
matrixes = [hilbert(n) for n in range(3,6)]

In [114]:
def regularisation_solution(a,b=None):
    if b is None:
        b = np.random.uniform(-100,100,size=(a.shape[1]))
    ans = pd.DataFrame(columns=["alpha","cond(a+alpha*E)","||x-x_alpha||"])
    l,u = lu(a)
    x = lu_solve(l,u,b)
    ans = ans.append(pd.Series([0,cond(a),0],index=ans.columns),True)
    E = np.identity(a.shape[0])
    for i in range(2,13,2):
        a_i = a + 10**(-i)*E
        l,u = lu(a_i)
        x_i = lu_solve(l,u,b)
        ans = ans.append(pd.Series([10**(-i),cond(a_i),norm(x_i-x)],index=ans.columns),True)
    return ans

In [125]:
regularisation_solution(matrixes[0]) #результат для матрицы Гильберта 3-го порядка

,alpha,cond(a+alpha*E),||x-x_alpha||
0,0.000000e+00,524.056778,0.000000
1,1.000000e-02,111.790091,16221.907429
2,1.000000e-04,505.291334,738.383159
3,1.000000e-06,523.862213,7.655746
4,1.000000e-08,524.054831,0.076586
5,1.000000e-10,524.056758,0.000766
6,1.000000e-12,524.056777,0.000008


In [127]:
regularisation_solution(matrixes[1]) #результат для матрицы Гильберта 4-го порядка

,alpha,cond(a+alpha*E),||x-x_alpha||
0,0.000000e+00,15513.738739,0.000000
1,1.000000e-02,149.575003,199263.868834
2,1.000000e-04,7627.334553,102257.412178
3,1.000000e-06,15354.963172,2058.729382
4,1.000000e-08,15512.134730,20.798036
5,1.000000e-10,15513.722697,0.208002
6,1.000000e-12,15513.738579,0.002080


In [128]:
regularisation_solution(matrixes[2]) #результат для матрицы Гильберта 5-го порядка

,alpha,cond(a+alpha*E),||x-x_alpha||
0,0.000000e+00,476607.250242,0.000000e+00
1,1.000000e-02,157.653234,1.485127e+07
2,1.000000e-04,15172.641273,1.438316e+07
3,1.000000e-06,365456.558250,3.464624e+06
4,1.000000e-08,475162.081827,4.504664e+04
5,1.000000e-10,476592.755044,4.518228e+02
6,1.000000e-12,476607.105284,4.518322e+00
